# Day 8
## Part 1

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (`acc`, `jmp`, or `nop`) and an argument (a signed number like `+4` or `-20`).

* `acc`: increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.

* `jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.

* `nop` stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:

```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

These instructions are visited in this order:

```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```

First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

***

### Notes
This is an interesting problem as it requires that each command runs once and only once. The reason is that the sequence of instructions (excluding the jumps) move top-down. Therefore, if a pointer returns to a command it has already ran before, then it'll most likely repeat the steps again.

We have the advantage of arrays and regex to solve this problem quickly and efficiently.

In [1]:
import re

instructions = list()

## Function to clean data
def parse_data(data):
    re_parser = r'(\w{3}) (\+\d+|\-\d+)'
    output = re.match(re_parser,data)
    return [output[1],int(output[2])]
    

## Open file and transform data into a list of lists
with open('inputs/day_8.txt') as reader:
    
    for instruction in reader:
        instructions.append(parse_data(instruction))
    
instructions[:3] # Sample

[['jmp', 336], ['jmp', 593], ['jmp', 121]]

In [2]:
# Let's also use the example data
sample_instructions = """nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6""".split("\n")

sample_instructions = [parse_data(s) for s in sample_instructions]
sample_instructions[:3]

[['nop', 0], ['acc', 1], ['jmp', 4]]

In [3]:
def instruction_parser(command, index, accumulator):
    
    command_typ = command[0]
    command_val = command[1]
    
    # Command manager
    if command_typ == "acc":
        return index + 1, command_val + accumulator
    
    elif command_typ == "nop":
        return index + 1, accumulator
    
    elif command_typ == "jmp":
        return index + command_val, accumulator
    else:
        return None, None


# Build the infinite loop finder
def infinite_finder(instructions):
    
    ## Initial Definition
    # A list that checks which indexes were already called
    instruction_tracker = []
    
    # Collects the accumulator value before the infinite call is made
    accumulator = 0
    
    # A flag to initiate a loop
    run_program = True
    
    # Initializer
    index = 0
    
    # A response flag (for Part 2)
    is_infinite = True
    
    while run_program:
        
        # Append to instruction that was already run
        instruction_tracker.append(index)
        
        # index should be within the range of the instructions array
        if index < len(instructions) and index >= 0:
            
            # Collect command and return value
            command = instructions[index]
            index, accumulator = instruction_parser(command, index, accumulator)

            # Check if the program has hit an index we already parsed?
            if index in instruction_tracker:
                run_program = False
                
        else:
            # We went through all the indexes successfully, therefore, not infinite!
            is_infinite = False
            run_program = False
            
    return accumulator, is_infinite

### Check the answer against sample

In [4]:
print("The answer by Advent of Code is 5.\nthe response by the function is {}.".format(infinite_finder(sample_instructions)[0]))

The answer by Advent of Code is 5.
the response by the function is 5.


In [5]:
infinite_finder(instructions)[0] # Answer

1766

---

## Part 2

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a `jmp` is supposed to be a `nop`, or a `nop` is supposed to be a `jmp`. (No `acc` instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

If you change the first instruction from `nop +0` to `jmp +0`, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from `jmp -4` to `nop -4`), the program terminates! The instructions are visited in this order:

```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```

After the last instruction (`acc +6`), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (`acc +1`, `acc +1`, `acc +6`).

Fix the program so that it terminates normally by changing exactly one `jmp` (to `nop`) or `nop` (to `jmp`).

What is the value of the accumulator after the program terminates?



In [6]:
# import deepcopy so we can copy lists without
# impacting parent list
from copy import deepcopy

# Build the infinite loop fixer
def infinite_fixer(instructions):
    
    ## Initial Definition
    
    # A list that identifies indexes that may need replacement
    targeted_instructions = []
    
    for i in range(len(instructions)):
        if instructions[i][0] == 'nop' or instructions[i][0] == 'jmp':
            targeted_instructions.append(i)
    
    # Assume the current set of instructions is infinite
    is_infinite = True
    
    # Copy list
    new_instructions = deepcopy(instructions)
    
    # Conduct a 'do-while' system on Python
    while True:
        
        # Run it without any changes
        accumulator, is_infinite = infinite_finder(new_instructions)
        if not is_infinite:
            # Break the loop once we've reached Nirvana (no loop, the program is fixed!)
            break
        
        # If we run out of all possibilities
        elif not targeted_instructions:
            accumulator = None
        
        # Let's modify stuff
        else:
            
            # Copy the old set of instructions
            new_instructions = deepcopy(instructions)
            
            # Access first target list
            i = targeted_instructions.pop(0)
            
            if new_instructions[i][0] == 'nop':
                new_instructions[i][0] = 'jmp'
            else:
                new_instructions[i][0] = 'nop'
    
    return accumulator # Final result after fix

In [7]:
infinite_fixer(sample_instructions) # Check sample answer

8

In [8]:
infinite_fixer(instructions) # Check answer

1639